In [1]:
import pyclops
from pyclops.core.loss_handler.chemical_loss_handler import ChemicalLossHandler

In [4]:
ChemicalLossHandler.from_pdb_file(pdb_file="/Users/bendupontjr/mphil_files/all_mphil_code/pyclops/examples/pdbs/CysAla5Cys.pdb", units_factor=1.0)

ValueError: Total number of keys in donor_atom_groups and acceptor_atom_groups must be exactly 4. Got 3 (donor: 2, acceptor: 2)